In [2]:
source("https://bioconductor.org/biocLite.R")

Bioconductor version 3.1 (BiocInstaller 1.18.4), ?biocLite for help


In [3]:
biocLite("MLSeq")

BioC_mirror: http://bioconductor.org
Using Bioconductor version 3.1 (BiocInstaller 1.18.4), R version 3.2.2.
Installing package(s) ‘MLSeq’
also installing the dependencies ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘brglm’, ‘GenomeInfoDb’, ‘XVector’, ‘futile.logger’, ‘AnnotationDbi’, ‘annotate’, ‘gridExtra’, ‘ggplot2’, ‘car’, ‘reshape2’, ‘foreach’, ‘BradleyTerry2’, ‘S4Vectors’, ‘IRanges’, ‘GenomicRanges’, ‘BiocParallel’, ‘genefilter’, ‘geneplotter’, ‘Hmisc’, ‘caret’, ‘DESeq2’, ‘Biobase’, ‘edgeR’




The downloaded source packages are in
	‘/tmp/RtmpWM8WqY/downloaded_packages’


Old packages: 'curl', 'devtools', 'evaluate', 'git2r', 'jsonlite', 'mime',
  'R6', 'Rcpp', 'xml2', 'XML', 'spatial'


## previous command took 5 minutes to complete

In [5]:
citation("MLSeq")


To cite package ‘MLSeq’ in publications use:

  Gokmen Zararsiz, Dincer Goksuluk, Selcuk Korkmaz, Vahap Eldem, Izzet
  Parug Duru, Turgay Unver and Ahmet Ozturk (2015). MLSeq: Machine
  learning interface for RNA-Seq data. R package version 1.6.0.

A BibTeX entry for LaTeX users is

  @Manual{,
    title = {MLSeq: Machine learning interface for RNA-Seq data},
    author = {Gokmen Zararsiz and Dincer Goksuluk and Selcuk Korkmaz and Vahap Eldem and Izzet Parug Duru and Turgay Unver and Ahmet Ozturk},
    year = {2015},
    note = {R package version 1.6.0},
  }

ATTENTION: This citation information has been auto-generated from the
package DESCRIPTION file and may need manual editing, see
‘help("citation")’.


In [4]:
library(MLSeq)

Loading required package: caret
Loading required package: lattice
Loading required package: ggplot2
Loading required package: DESeq2
Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:stats’:

    xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, intersect,
    is.unsorted, lapply, Map, mapply, match, mget, order, paste, pmax,
    pmax.int, pmin, pmin.int, Position, rank, rbind, Reduce, rep.int,
    rownames, sapply, setdiff, sort, table, tapply, un

In [6]:
filepath = system.file("extdata/cervical.txt", package = "MLSeq")
filepath

[1] "/home/daniel/R/x86_64-pc-linux-gnu-library/3.2/MLSeq/extdata/cervical.txt"

In [7]:
cervical = read.table(filepath, header = TRUE)

In [8]:
head(cervical[, 1:5])

,N1,N2,N3,N4,N5
let-7a,865,810,5505,6692,1456
let-7a*,3,12,30,73,6
let-7b,975,2790,4912,24286,1759
let-7b*,15,18,27,119,11
let-7c,828,1251,2973,6413,713
let-7c*,0,0,0,1,0


In [9]:
class(cervical)

[1] "data.frame"

In [10]:
dim(cervical)

[1] 714  58

First 29 columns of the data contain the miRNA mapped counts of non-tumor samples, while the last 29 comlumns contain the count information of tumor samples. We need to create a class label in order to apply classification models.

In [11]:
class = data.frame(condition = factor(rep(c("N", "T"), c(29, 29))))
as.factor(class[, 1])

[1] N N N N N N N N N N N N N N N N N N N N N N N N N N N N N T T T T T T T T T
[39] T T T T T T T T T T T T T T T T T T T T
Levels: N T

For simplicity, we can work with a subset of cervical data using the first 150 features.

In [12]:
data = cervical[c(1:150), ]

Now we can split the data into training and test sets. The thraining set can be used to build classification models, and the test set can be used to assess the performance of each model. We use the set.seed function to specify initiala values of a random-number seed and use the sample function for selection.

In [13]:
nTest = ceiling(ncol(data) * 0.2)
set.seed(12345)
ind = sample(ncol(data), nTest, FALSE)

Now, training and test sets can be created based on this sampling process.

In [18]:
data.train = data[, -ind]
data.train = as.matrix(data.train + 1)
classtr = data.frame(condition = class[-ind, ])

data.test = data[, ind]
data.test = as.matrix(data.test + 1)
classts = data.frame(condition = class[ind, ])

Now we have 46 samples which will be used to train the classification models and have 12 remaining samples to test model performance.

In [19]:
dim(data.train)

[1] 150  46

In [20]:
dim(data.test)

[1] 150  12

In [21]:
data.trainS4 = DESeqDataSetFromMatrix(countData = data.train, colData = classtr, formula(~condition))
data.trainS4 = DESeq(data.trainS4, fitType = "local")
data.trainS4


converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
-- replacing outliers and refitting for 15 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)
estimating dispersions
fitting model and testing


class: DESeqDataSet 
dim: 150 46 
exptData(0):
assays(5): counts mu cooks replaceCounts replaceCooks
rownames(150): let-7a let-7a* ... miR-18a* miR-18b
rowRanges metadata column names(28): baseMean baseVar ... maxCooks
  replace
colnames(46): 1 2 ... 45 46
colData names(3): condition sizeFactor replaceable

In [22]:
data.testS4 = DESeqDataSetFromMatrix(countData = data.test, colData = classts, formula(~condition))
data.testS4 = DESeq(data.testS4, fitType = "local")
data.testS4

converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


class: DESeqDataSet 
dim: 150 12 
exptData(0):
assays(3): counts mu cooks
rownames(150): let-7a let-7a* ... miR-18a* miR-18b
rowRanges metadata column names(27): baseMean baseVar ... deviance
  maxCooks
colnames(12): 1 2 ... 11 12
colData names(2): condition sizeFactor

In [23]:
svm = classify(data = data.trainS4, method = "svm", normalize = "deseq", deseqTransform = "vst", cv = 5, rpt = 3, ref = "T")

svm

found already estimated dispersions, replacing these
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates


1 package is needed for this model and is not installed. (kernlab). Would you like to try to install it now?

ERROR: Error in checkInstall(models$library): 


ERROR: Error in eval(expr, envir, enclos): object 'svm' not found


In [25]:
install.packages("kernlab", repos='http://archive.linux.duke.edu/cran/')

Installing package into ‘/home/daniel/R/x86_64-pc-linux-gnu-library/3.2’
(as ‘lib’ is unspecified)



The downloaded source packages are in
	‘/tmp/RtmpWM8WqY/downloaded_packages’


In [27]:
install.packages("e1071", repos='http://archive.linux.duke.edu/cran/')

Installing package into ‘/home/daniel/R/x86_64-pc-linux-gnu-library/3.2’
(as ‘lib’ is unspecified)



The downloaded source packages are in
	‘/tmp/RtmpWM8WqY/downloaded_packages’


In [28]:
svm = classify(data = data.trainS4, method = "svm", normalize = "deseq", deseqTransform = "vst", cv = 5, rpt = 3, ref = "T")

svm

found already estimated dispersions, replacing these
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates



  An object of class  MLSeq 

            Method  :  svm 

       Accuracy(%)  :  93.48 
    Sensitivity(%)  :  100 
    Specificity(%)  :  86.96 

  Reference Class   :  T 


After running the code given above, we obtain the results in MLSeq class.  SVM successfully fits
a model with 97.8% true classifcation accuracy by misclassifying only one non-tumor sample.


"svm" object also stores information about model training and the parameters used to build
this model.

In [29]:
getSlots("MLSeq")

method    deseqTransform     normalization      confusionMat 
      "character"       "character"       "character" "confusionMatrix" 
          trained               ref 
          "train"       "character"

In [30]:
trained(svm)

Support Vector Machines with Radial Basis Function Kernel 

 46 samples
150 predictors
  2 classes: 'T', 'N' 

No pre-processing
Resampling: Cross-Validated (5 fold, repeated 3 times) 
Summary of sample sizes: 37, 37, 37, 37, 36, 37, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa      Accuracy SD  Kappa SD 
  0.25  0.8748148  0.7534359  0.11792344   0.2261951
  0.50  0.9044444  0.8114907  0.10164433   0.1945290
  1.00  0.8985185  0.8002087  0.09811869   0.1872383

Tuning parameter 'sigma' was held constant at a value of 0.003875898
Accuracy was used to select the optimal model using  the largest value.
The final values used for the model were sigma = 0.003875898 and C = 0.5. 

Now let's build a Random Forest model

In [32]:
rf <- classify(data = data.trainS4, method = "randomforest", normalize = "deseq", deseqTransform = "vst", cv = 5, rpt= 3, ref = "T")

rf

found already estimated dispersions, replacing these
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates



  An object of class  MLSeq 

            Method  :  randomforest 

       Accuracy(%)  :  100 
    Sensitivity(%)  :  100 
    Specificity(%)  :  100 

  Reference Class   :  T 


# Prediction of new samples from the test set

In [33]:
pred.svm = predictClassify(svm, data.testS4)

pred.svm

found already estimated dispersions, replacing these
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates


[1] T T T T T T N N T T N N
Levels: T N

In [34]:
pred.rf = predictClassify(rf, data.testS4)

pred.rf

found already estimated dispersions, replacing these
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates


[1] T T T T N T N N T T N N
Levels: T N

In [35]:
table(pred.svm, relevel(data.testS4$condition, 2))

table(pred.rf, relevel(data.testS4$condition, 2))

        
pred.svm T N
       T 6 2
       N 0 4

       
pred.rf T N
      T 6 1
      N 0 5

In [36]:
sessionInfo()

R version 3.2.2 (2015-08-14)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 14.04.3 LTS

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] kernlab_0.9-22            MLSeq_1.6.0              
 [3] edgeR_3.10.2              randomForest_4.6-10      
 [5] limma_3.24.15             Biobase_2.28.0           
 [7] DESeq2_1.8.1              RcppArmadillo_0.5.500.2.0
 [9] Rcpp_0.12.0               GenomicRanges_1.20.6     
[11] GenomeInfoDb_1.4.2        IRanges_2.2.7            
[13] S4Vectors_0.6.5           BiocGenerics_

In [37]:
confusionMatrix(table(pred.svm, relevel(data.testS4$condition, 2)))

Confusion Matrix and Statistics

        
pred.svm T N
       T 6 2
       N 0 4
                                          
               Accuracy : 0.8333          
                 95% CI : (0.5159, 0.9791)
    No Information Rate : 0.5             
    P-Value [Acc > NIR] : 0.01929         
                                          
                  Kappa : 0.6667          
 Mcnemar's Test P-Value : 0.47950         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.6667          
         Pos Pred Value : 0.7500          
         Neg Pred Value : 1.0000          
             Prevalence : 0.5000          
         Detection Rate : 0.5000          
   Detection Prevalence : 0.6667          
      Balanced Accuracy : 0.8333          
                                          
       'Positive' Class : T               
                                          